In [77]:
import os
import sys
import urllib.request
import datetime
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import UnexpectedAlertPresentException

In [78]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", "jL0_Mj8EcTrEmAvqNflt")
    req.add_header("X-Naver-Client-Secret", "XjFsAbKpCH")
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [79]:
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [81]:
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    link = post['link']
    
    jsonResult.append({'title':title, 'description': description, 'link': link})
        
    return

In [80]:
def makeJson():

    jsonResult = []

    # 'news', 'blog', 'cafearticle', 'kin'

    sNode = 'kin'
    search_text = '전자담배'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        
        if (nStart > 1000):
            break;
            
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult,
                        indent=4, sort_keys=True,
                        ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver_%s.json SAVED' % (search_text, sNode))
    
    return search_text

In [82]:
def makeLinkList(keyword):
    items = json.load(open(keyword + "_naver_kin.json", "r", encoding="utf-8"))
    links = []
    
    i = 1
    for item in items:
        print(str(i) + " : " + item["link"])
        links.append(item["link"])
        

        if i == 5:
            break
          
        
        i += 1
    
    return links

In [85]:
def textCrawling(links, keyword):
    
    wfile = open(os.getcwd() + "/" + keyword +"_naver_kin.txt", mode='w', encoding='utf-8')
    
    idx = 1
    for link in links:
        try:
            driver = webdriver.Chrome("C:/Users/comdori/Downloads/chromedriver_win32/chromedriver.exe")
            driver.get(link)

            print("\nnaver_kin_" + str(idx) + "\n")
            
            html = driver.page_source

            bs = BeautifulSoup(html, 'html.parser')
            # print(bs.prettify())

            title = bs.select("#content > div.question-content > div > div.c-heading._questionContentsArea > div.c-heading__title > div > div")
            question = bs.select("#content > div.question-content > div > div.c-heading._questionContentsArea.c-heading--default-old > div.c-heading__content")
            answer = bs.find_all('div', {'class':'c-heading-answer__content-user'})

            wfile.write("\nnaver_kin_" + str(idx) + "\n\n")

            # title
            for t in title:
                contents = t.get_text().strip()
                print(contents)
                wfile.write(contents)

            # question
            for q in question:
                contents = q.get_text().strip()
                print(contents)
                wfile.write(contents)

            # answer
            for a in answer:
                contents = a.get_text().strip()
                print(contents)
                wfile.write(contents)

            wfile.write("\n")
        
        except UnexpectedAlertPresentException as e:
            print(type(e))
            driver.switch_to.alert.accept()
            print("Alert accepted")
        
        except Exception as e:
            print(type(e))
        
        finally:
            driver.close()
            idx = idx + 1
    
    wfile.close()

In [86]:
def main():
    keyword = makeJson()
    links = makeLinkList(keyword)
    #links = ['https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=4010505&docId=311351460&qb=7KCE7J6Q64u067Cw&enc=utf8&section=kin.qna&rank=3&search_sort=0&spq=0']
    result = textCrawling(links, keyword)
    
main()

[2019-05-15 23:14:58.740005] Url Request Success
[2019-05-15 23:14:59.027101] Url Request Success
[2019-05-15 23:14:59.344457] Url Request Success
[2019-05-15 23:14:59.664243] Url Request Success
[2019-05-15 23:14:59.954070] Url Request Success
[2019-05-15 23:15:00.282016] Url Request Success
[2019-05-15 23:15:00.580846] Url Request Success
[2019-05-15 23:15:00.897664] Url Request Success
[2019-05-15 23:15:01.191459] Url Request Success
[2019-05-15 23:15:01.492809] Url Request Success
전자담배_naver_kin.json SAVED
1 : https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=40502&docId=326811158&qb=7KCE7J6Q64u067Cw&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0
2 : https://kin.naver.com/qna/detail.nhn?d1id=6&dirId=60105&docId=327109559&qb=7KCE7J6Q64u067Cw&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=0
3 : https://kin.naver.com/qna/detail.nhn?d1id=4&dirId=4010505&docId=311351460&qb=7KCE7J6Q64u067Cw&enc=utf8&section=kin.qna&rank=3&search_sort=0&spq=0
4 : https://kin.naver.com/qna/detail.n